## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
import ipywidgets as ipyw
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [3]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [4]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3
0,-0.132894,0.015893,-0.013382,0.041156
1,-0.142732,0.009482,-0.026800,0.016065
2,-0.065428,-0.017965,0.101164,-0.060902
3,-0.078180,-0.015748,0.050627,-0.128463
4,-0.095028,-0.060968,-0.134726,0.027579


In [5]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USS0006K01S', u'US1COPK0005', u'US1COJF0281', u'USS0005K05S', u'USC00050214']


,0,1,2,3
station,,,,
USS0006K01S,-0.132894,0.015893,-0.013382,0.041156
US1COPK0005,-0.142732,0.009482,-0.026800,0.016065
US1COJF0281,-0.065428,-0.017965,0.101164,-0.060902
USS0005K05S,-0.078180,-0.015748,0.050627,-0.128463
USC00050214,-0.095028,-0.060968,-0.134726,0.027579


In [6]:
table=Eig.join(Stations,how='left')
table.head(4)

,0,1,2,3,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
station,,,,,,,,,,,,
USS0006K01S,-0.132894,0.015893,-0.013382,0.041156,39.3667,-106.0667,3474.7,O,HOOSIER PASS,NaN,NaN,NaN
US1COPK0005,-0.142732,0.009482,-0.026800,0.016065,39.3303,-106.0611,3225.1,O,ALMA 3.1 N,NaN,NaN,NaN
US1COJF0281,-0.065428,-0.017965,0.101164,-0.060902,39.7674,-105.2262,1805.0,O,GOLDEN 2.0 NNW,NaN,NaN,NaN
USS0005K05S,-0.078180,-0.015748,0.050627,-0.128463,39.6667,-105.9000,3474.7,O,LOVELAND BASIN,NaN,NaN,NaN


In [7]:
table=table[['name','latitude','longitude','elevation',0,1,2,3]]#,4,5,6,7]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USS0006K01S,HOOSIER PASS,39.3667,-106.0667,3474.7,-0.132894,0.015893,-0.013382,0.041156
US1COPK0005,ALMA 3.1 N,39.3303,-106.0611,3225.1,-0.142732,0.009482,-0.026800,0.016065
US1COJF0281,GOLDEN 2.0 NNW,39.7674,-105.2262,1805.0,-0.065428,-0.017965,0.101164,-0.060902
USS0005K05S,LOVELAND BASIN,39.6667,-105.9000,3474.7,-0.078180,-0.015748,0.050627,-0.128463


In [8]:
type(table)

pandas.core.frame.DataFrame

In [9]:
table.index

Index([u'USS0006K01S', u'US1COPK0005', u'US1COJF0281', u'USS0005K05S',
       u'USC00050214', u'USC00052633', u'US1COGF0004', u'US1COGF0003',
       u'US1COGF0002', u'US1COGF0001',
       ...
       u'USW00093009', u'USS0006K39S', u'USS0006K06S', u'USC00054762',
       u'USC00053146', u'USC00053387', u'USS0006K10S', u'USC00050825',
       u'US1COEG0021', u'US1COLK0022'],
      dtype='object', name=u'station', length=201)

In [10]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [11]:
min_lat,max_lat,min_long,max_long = box = (39.1014, 39.8227, -108.7339, -105.1171)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r
lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
#         print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    
# ipyw.VBox([m, label])

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [12]:
# map = ipyl.Map(center=[53.88, 27.45], zoom=4)
# label = ipyw.Label(layout=ipyw.Layout(width='100%'))

# # geojson layer with hover handler
# with open('./europe_110.geo.json') as f:
#     data = json.load(f)
# for feature in data['features']:
#     feature['properties']['style'] = {
#         'color': 'grey',
#         'weight': 1,
#         'fillColor': 'grey',
#         'fillOpacity': 0.5
#     }
# layer = ipyl.GeoJSON(data=data, hover_style={'fillColor': 'red'})

# def hover_handler(event=None, id=None, properties=None):
#     label.value = properties['geounit']

# layer.on_hover(hover_handler)
# map.add_layer(layer)


# ipyw.VBox([map, label])